# Perform pdf ocr and save images
 
**directory:** 
* /content/drive/Shareddrives/Data Science Team folder-open/Eurocoorp/

**ImageDirectory**
* Images/

**Image Name format**
* 100page-0.jpg 
* *pdf file name + pdfpage count*

sample: */content/drive/Shareddrives/Data Science Team folder-open/Eurocoorp/Images/100page-0.jpg*

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 62 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,240 kB/s)
debconf: unable to initi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# traversing all files
folderpath = '/content/drive/Shareddrives/Data Science Team folder-open/Eurocoorp/'
import os
from os import walk

filenames = next(walk(folderpath), (None, None, []))[2]
len(filenames)

242

In [ ]:
#create folder to save images
imgpath = folderpath+ r'200dpi/' 
if not os.path.exists(imgpath):
    os.makedirs(imgpath)
    print("DONE")

DONE


In [ ]:
# converting all pdf files into images
!pip install PyMuPDF
import fitz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 3.4 MB/s 


In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(filenames))):
  pdffile = folderpath+filenames[i]
  doc = fitz.open(pdffile)
  for page in doc:
    pix = page.get_pixmap(dpi = 200)
    pix.save(imgpath+filenames[i][:-4]+"page-%i.jpg" % page.number)

100%|██████████| 242/242 [01:26<00:00,  2.80it/s]


In [ ]:
# Performing OCR using tessearct

imgnames = next(walk(imgpath), (None, None, []))[2]
imgnames[:10]

['231page-0.jpg',
 '231page-1.jpg',
 '147page-0.jpg',
 '147page-1.jpg',
 '63page-0.jpg',
 '63page-1.jpg',
 '62page-0.jpg',
 '62page-1.jpg',
 '215page-0.jpg',
 '215page-1.jpg']

In [ ]:
# deleting the irrelevant png file extensions, if any
import os, glob
for filename in glob.glob(imgpath+"*png"):
    os.remove(filename)
    print(filename)

In [ ]:
import numpy as np
import pytesseract
from PIL import Image
import pandas as pd
import re

In [ ]:
#create folder to save csv
csvpath = folderpath+ r'Csv/' 
if not os.path.exists(csvpath):
    os.makedirs(csvpath)
    print("DONE")


In [ ]:
for i in tqdm(range(len(imgnames))):
  path = imgpath +imgnames[i]
  regex = r'[0-9]*$'
  m =  re.findall(r'\d+',path)
  sample = pytesseract.image_to_data(Image.open(path),lang='eng', output_type='data.frame')
  sample.to_csv(csvpath+m[0]+'page-'+m[1]+'.csv', index=False)

100%|██████████| 473/473 [29:06<00:00,  3.69s/it]


## Annotating the key values
The keys can be labelled because they are fixed and the names do not change.  


In [ ]:
# dict of keys to be annotated
keys = {
'PRODUTTORE':'produttore_key',
'CLIENTE': 'cliente_key',
'Unit. Loc.': 'unitloc_key',
'CF Produttore':'cfproduttore_key',
'Operatore':'operatore_key',
'Email Cliente':'emailcliente_key',
'Referente':'referente_key',
'Email Produttore':'emailproduttore_key',
'Telefono':'telefono_key',
'Data Richiesta':'datarichiesta_key',
'Orario richiesta':'orariorichiesta_key',
'Mattina':'mattina_key',
'Pomeriggio':'pomeriggio_key',
'CER':'cer_key',
'DESCRIZIONE':'descrizione_key',
'SF':'sf_key',
'Classe Pericolo:':'classpericolo_key',
'Confezionamento:':'confezionamento_key',
'Destinazione:':'destinazione_key',
'Destino:':'destino_key'}


In [ ]:
csvnames = next(walk(csvpath), (None, None, []))[2]

In [ ]:
temp_csvpath = csvpath+csvnames[0]
df = pd.read_csv(temp_csvpath)
df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1653,2339,-1,NaN
1,2,1,1,0,0,0,759,40,111,43,-1,NaN
2,3,1,1,1,0,0,759,40,111,43,-1,NaN
3,4,1,1,1,1,0,759,40,111,43,-1,NaN
4,5,1,1,1,1,1,759,40,111,43,14,af


In [ ]:
#Creating new directory to store updated csv files.

csvUpdatedpath = csvpath+ 'Updated/' 
if not os.path.exists(csvUpdatedpath):
    os.makedirs(csvUpdatedpath)
    print("DONE")

DONE


In [ ]:
csvpath

'/content/drive/Shareddrives/Data Science Team folder-open/Eurocoorp/Csv/'

In [ ]:
from tqdm import tqdm
for name in tqdm(range(len(csvnames))):
  #creating df for each csv 
  temp_csvpath = csvpath+csvnames[name]
  df = pd.read_csv(temp_csvpath)
  l = df.shape[0]
  #initializing new column with default value as 'others'
  df['label'] = 'others'

  for i in range(l):
    #Handling nan values
    if pd.isna(df['text'].iat[i]) == True:
      pass
      # df['label'].iat[i] = 'others'
    
    #Handling double valued keys like - 'CF Produttore'
    else:
      #for CF Produttore
      if df['text'].iat[i] in ['CF']:
        if df['text'].iat[i+1] in ['Produttore']:
          df['label'].iat[i], df['label'].iat[i+1] = 'cfproduttore_key','cfproduttore_key'
      #for Classe Pericolo
      if df['text'].iat[i] in ['Classe']:
        if df['text'].iat[i+1] in ['Pericolo']:
          df['label'].iat[i], df['label'].iat[i+1] = 'classpericolo_key','classpericolo_key'
      #for Data Richiesta
      if df['text'].iat[i] in ['Data']:
        if df['text'].iat[i+1] in ['Richiesta']:
          df['label'].iat[i], df['label'].iat[i+1] = 'datarichiesta_key','datarichiesta_key'
      #for Email Cliente
      if df['text'].iat[i] in ['Email']:
        if df['text'].iat[i+1] in ['Cliente']:
          df['label'].iat[i], df['label'].iat[i+1] = 'emailcliente_key','emailcliente_key'
      #for Email Produttore
      if df['text'].iat[i] in ['Email']:
        if df['text'].iat[i+1] in ['Produttore']:
          df['label'].iat[i], df['label'].iat[i+1] = 'emailproduttore_key','emailproduttore_key'
      #for Orario richiesta
      if df['text'].iat[i] in ['Orario']:
        if df['text'].iat[i+1] in ['richiesta']:
          df['label'].iat[i], df['label'].iat[i+1] = 'orariorichiesta_key','orariorichiesta_key'
      #for Unit. Loc. 
      if df['text'].iat[i] in ['Unit.']:
        if df['text'].iat[i+1] in ['Loc.']:
          df['label'].iat[i], df['label'].iat[i+1] = 'unitloc_key','unitloc_key'
    
    # Handling remaining keys
    for key in keys:
      if df['text'].iat[i] == key:
        df['label'].iat[i]= keys.get(key)
        pass
  df.to_csv(csvpath+ 'Updated/'+ csvnames[name])

100%|██████████| 473/473 [00:34<00:00, 13.87it/s]


In [ ]:
df.to_csv('/content/temp2.csv') #/content/temp.csv

In [ ]:
df.to_csv('/content/temp.csv')

In [ ]:
#iterating the csv files:
csvnames = next(walk(csvpath), (None, None, []))[2]
for i in tqdm(range(len(csvnames))):
  temp_csvpath = csvpath+ csvnames[0]
  df = pd.read_csv(temp_csvpath)
  df.head()
  break

  0%|          | 0/474 [00:00<?, ?it/s]
